# 训练领域模型--医疗模型

分三阶段：

第一阶段：PT(continue PreTraining)增量预训练，在海量领域文档数据上二次预训练LLaMA模型，如有需要可以扩充领域词表，比如医疗领域词表；
第二阶段：SFT(Supervised Fine-tuning)有监督微调，构造指令微调数据集，在第一阶段的预训练模型基础上做指令精调；
第三阶段：RM(Reward Model)奖励模型，和基于人类反馈的强化学习(RLHF)。


## PT(continue PreTraining)增量预训练

使用百科类文档类数据集，用来在领域数据集上增量预训练或二次预训练，期望能把领域知识注入给模型，以医疗领域为例，希望增量预训练，能让模型理解感冒的症状、病因、治疗药品、治疗方法、药品疗效等知识，便于后续的SFT监督微调能激活这些内在知识。

这里说明一点，像GPT3、LLaMA这样的大模型理论上是可以从增量预训练中获益，但增量预训练需要满足两个要求：1）高质量的预训练样本；2）较大的计算资源，显存要求高，即使是用LoRA技术，也要满足block_size=1024或2048长度的文本加载到显存中。

其次，如果你的项目用到的数据是模型预训练中已经使用了的，如维基百科、ArXiv等LLaMA模型预训练用了的，则这些数据是没有必要再喂给LLaMA增量预训练，而且预训练样本的质量如果不够高，也可能会损害原模型的生成能力。

tips：PT阶段是可选项，请慎重处理。

In [ ]:
!git clone --depth 1 git@github.com:shibing624/textgen.git
!cd textgen
!python setup install

In [ ]:
!cd examples/llama

In [ ]:
import os
import sys

import pandas as pd
from datasets import load_dataset, load_from_disk
from loguru import logger

sys.path.append('../..')
from textgen import LlamaModel


def load_pt_data(data, train_limit_size=10000):
    if data.endswith('.json') or data.endswith('.jsonl'):
        dataset = load_dataset("json", data_files=data)
    elif os.path.isdir(data):
        dataset = load_from_disk(data)
    else:
        dataset = load_dataset(data, 'pretrain')
    logger.debug(dataset)

    if train_limit_size > 0:
        dataset_sample = dataset["train"].shuffle(seed=42).select(range(train_limit_size))
    else:
        dataset_sample = dataset["train"].shuffle(seed=42)
    all_datasets = dataset_sample
    logger.debug(f"all_datasets size:{len(all_datasets)}, first line: {next(iter(all_datasets))}")
    data = []
    for example in all_datasets:
        data.append(example['text'])
    logger.info(f"total data size:{len(data)}, head: {data[:3]}")
    return data

In [ ]:
logger.info('Loading data...')
model_args = {
    "is_pretraining": True,
    "use_peft": True,
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "per_device_train_batch_size": 2,
    "eval_batch_size": 2,
    "num_train_epochs": 2,
    "output_dir": './outputs-pretraining-medical/',
    "resume_from_checkpoint": './outputs-pretraining-medical/',
    "eval_steps": 50,
    "save_steps": 100,
}
model = LlamaModel('llama', 'shibing624/chinese-llama-plus-13b-hf', args=model_args)
model

In [ ]:
print(model.predict(["给出三个保持健康的秘诀。"]))
print(model.predict(["乙肝和丙肝的区别"]))
print(model.predict(["什么补肾最好"]))

In [ ]:
train_data = load_pt_data('shibing624/medical', 10000)
logger.debug(f'train_data, size: {len(train_data)}, head top3: {train_data[:3]}')
train_df = pd.DataFrame(train_data, columns=["text"])
train_df

In [ ]:
eval_df = train_df[:10]
eval_df

In [ ]:
model.train_model(train_df, eval_data=eval_df)

In [ ]:
print(model.predict(["给出三个保持健康的秘诀。"]))
print(model.predict(["乙肝和丙肝的区别"]))
print(model.predict(["什么补肾最好"]))

## SFT(Supervised Fine-tuning)有监督微调

构造指令数据集，在第一阶段的预训练模型基础上做指令精调。

如果是LoRA的增量预训练，需要先把LoRA权重merge到base model, 使用merge脚本完成合并操作，再SFT；
如果没有做增量预训练，则直接在底座模型上SFT就可以。


<img src=../../docs/rlhf.png height=500 />


以下脚本是可选的，需要就执行，不需要注释掉：

In [ ]:
# merge lora to base model
!python -m textgen/llama/merge_peft_adapter.py \
    --base_model_name_or_path shibing624/chinese-llama-plus-13b-hf \
    --peft_model_path ./outputs-pretraining-medical/ \
    --output_type huggingface --output_dir merged_pt

基于llama-13b-plus底座模型或者merge_pt的底座模型，SFT监督训练生成llama-13b-sft:

In [ ]:
from datasets import load_dataset, concatenate_datasets

def load_sft_data(debug=True):
    raw_dataset = load_dataset('shibing624/medical', 'finetune')
    logger.debug(raw_dataset)

    all_datasets = []
    if debug:
        all_datasets.append(raw_dataset["train"].shuffle().select(range(1000)))
    else:
        all_datasets.append(raw_dataset["train"])
    # sft 阶段，除了加载医疗训练集外，为了避免模型灾难遗忘，需要强化其正常对话能力，此处加载中英文alpaca、中文belle数据集
    dataset_names = ["shibing624/alpaca-zh", "yahma/alpaca-cleaned", "BelleGroup/train_0.5M_CN", "BelleGroup/generated_chat_0.4M"]
    for name in dataset_names:
        raw_dataset = load_dataset(name)
        ds = raw_dataset["train"]
        if debug:
            ds = ds.shuffle().select(range(1000))
        all_datasets.append(ds)
    all_datasets = concatenate_datasets(all_datasets)
    data = []
    logger.debug(f"all_datasets size:{len(all_datasets)}, first line: {next(iter(all_datasets))}")
    for example in all_datasets:
        instruction, intput_str, output_str = example["instruction"], example["input"], example["output"]
        data.append([instruction, intput_str, output_str])
    logger.info(f"total data size:{len(data)}, head: {data[:3]}")
    return data



In [ ]:
model_name = 'merged_pt' if os.path.exists('merged_pt') else 'shibing624/chinese-llama-plus-13b-hf'

In [ ]:
# fine-tune Llama model
model_args = {
    "use_peft": True,
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 256,
    "max_length": 256,
    "per_device_train_batch_size": 8,
    "eval_batch_size": 8,
    "num_train_epochs": 2,
    "is_train_on_prompt": False,
    "output_dir": './outputs-sft/',
    "resume_from_checkpoint": './outputs-llama-13b-sft/',
    "eval_steps": 100,
    "save_steps": 500,
}
model = LlamaModel('llama', model_name, args=model_args)
model

In [ ]:
print(model.predict(["给出三个保持健康的秘诀。"]))
print(model.predict(["乙肝和丙肝的区别"]))
print(model.predict(["什么补肾最好"]))

In [ ]:
train_data = load_sft_data(debug=True)
logger.debug('train_data: {}'.format(train_data[:10]))
train_df = pd.DataFrame(train_data, columns=["instruction", "input", "output"])
train_df

In [ ]:
eval_df = train_df[:10]
eval_df

In [ ]:
model.train_model(train_df, eval_data=eval_df)

In [ ]:
def generate_prompt(instruction):
    return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n\n"""

print(model.predict([generate_prompt("给出三个保持健康的秘诀。")]))
print(model.predict([generate_prompt("乙肝和丙肝的区别")]))
print(model.predict(["什么补肾最好"]))
print(model.predict([generate_prompt("防中风还可吃什么药？")]))

## 奖励模型和人类偏好
此处需要训练两个模型：RM(Reward Model)奖励模型和RL模型（PPO）

原则上，我们可以直接用人类标注来对模型做 RLHF 微调。然而，这将需要我们给人类发送一些样本，在每轮优化后计分。这是贵且慢的，因为收敛需要的训练样本量大，而人类阅读和标注的速度有限。

一个比直接反馈更好的策略是，在进入 RL 循环之前用人类标注集来训练一个奖励模型RM。奖励模型的目的是模拟人类对文本的打分。构建奖励模型有许多能用的策略: 最直接的便是预测标注 (比如根据好与坏，输出比分或者布尔值)。最佳实践是，预测结果的排序，即对每个 prompt (输入文本) 对应的两个结果 (yk, yj)，模型预测人类标注的比分哪个更高。

RM模型是通过人工标注SFT模型的打分结果来训练的，目的是取代人工打分，本质是个回归模型，用来对齐人类偏好，主要是"HHH"原则，具体是"helpful, honest, harmless"。

RM模型用来及时对生成模型的每条结果打分，正向的得分就奖励，负向的得分就惩罚。

#### 基于人类反馈的强化学习(RLHF)
现在我们手头有了微调的语言模型和奖励模型，可以开始执行 RL 循环了: 这个过程大致分为三步

1. 生成对 prompt (输入文本) 的反馈。
2. 用奖励模型来对反馈评分。
3. 对评分，进行一轮策略优化的强化学习。

<img src=https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/blog/stackllama/trl_loop.png height=400 />



基于llama-13b-sft模型，使用对话样本对，训练得到llama-13b-sft-rm:

In [ ]:
!torchrun --nnodes 1  --nproc_per_node 8 reward_modeling.py --model_name=<LLAMA_SFT_MODEL>

基于llama-13b-sft-rm模型 RL 微调训练 llama-13b-sft模型，得到llama-13b-sft-rl:

In [ ]:
!accelerate launch --multi_gpu --num_machines 1  --num_processes 8 rl_training.py --log_with=wandb --model_name=<LLAMA_SFT_MODEL> --reward_model_name=<LLAMA_SFT_RM_MODEL> --adafactor=False --tokenizer_name=<LLAMA_TOKENIZER> --save_freq=100 --output_max_length=128 --batch_size=8 --gradient_accumulation_steps=8 --batched_gen=True --ppo_epochs=4 --seed=0 --learning_rate=1.4e-5 --early_stopping=True --output_dir=llama-se-rl-finetune-128-8-8-1.4e-5_adam

注意：使用lora训练，每一步的lora权重都要合并到base模型中，再进行下一步。

合并方法：

In [ ]:
!python -m textgen/llama/merge_peft_adapter.py \
    --base_model_name_or_path xxx \
    --peft_model_path yyy \
    --output_type huggingface --output_dir xxx